In [1]:
from baseline_transformer_MORPH_conv import *
import copy

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())  # Renvoie True si un GPU est disponible

batch_size=16
epochs=10

False


In [3]:
# Load data using the load_data_1 function
sentences, morphs = load_data("UD_French-Sequoia/fr_sequoia-ud-train.conllu")

# Create character and tag mappings
char_counts = Counter(char for sentence in sentences for word in sentence for char in word)
char_to_ix = {char: i for i, char in enumerate(char_counts, start=2)}
char_to_ix['<PAD>'], char_to_ix['<UNK>'] = 0, 1  # Padding and unknown character

morph_counts = Counter(tag for tags in morphs for tag in tags)
morph_to_ix = {tag: i for i, tag in enumerate(morph_counts)}

max_word_len = max(len(word) for sentence in sentences for word in sentence)

# Now load the data in the desired format using the load_data function
train_sentences, train_morph = load_data_preprocess("UD_French-Sequoia/fr_sequoia-ud-train.conllu", char_to_ix, max_word_len)
validation_sentences, validation_morph = load_data_preprocess("UD_French-Sequoia/fr_sequoia-ud-dev.conllu", char_to_ix, max_word_len)

# Rest of your code for Dataset, DataLoader, Model initialization, etc.

# Dataset and DataLoader
dataset = MORPHDataset(train_sentences, train_morph, morph_to_ix, max_word_len,char_to_ix)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

validation_dataset = MORPHDataset(validation_sentences, validation_morph, morph_to_ix, max_word_len,char_to_ix)
validation_data_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [4]:
# Model initialization
num_chars = len(char_to_ix)
char_embedding_dim = 512
num_filters = 512
kernel_size = 20
nhead = 4
nhid = 1024
nlayers = 3
tagset_size = len(morph_to_ix)

model = MorphTransformerModel(num_chars, char_embedding_dim, num_filters, kernel_size, nhead, nhid, nlayers, tagset_size)

# Loss and Optimizer
loss_function = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


C:\Users\Thomas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<h2> Training

In [5]:
patience = 5  # Nombre d'époques à attendre après la dernière amélioration de la loss de validation
best_val_accuracy = 0
epochs_no_improve = 0

#Training
for epoch in range(epochs): 
    model.train()
    model.to(device)  # Déplacer le modèle sur le GPU si disponible
    total_loss = 0
    for sentence_in, targets in data_loader:
        sentence_in, targets = sentence_in.to(device), targets.to(device)  # Déplacer les données sur le périphérique
        optimizer.zero_grad()
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores.view(-1, len(morph_to_ix)), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Utiliser la fonction modifiée pour évaluer la validation loss et l'accuracy
    val_loss, val_accuracy = evaluate_model(model, validation_data_loader, loss_function,device, morph_to_ix)
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = copy.deepcopy(model)
        print(f"best model accuracy: {best_val_accuracy:.4f}")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    # Arrêt précoce si aucune amélioration
    if epochs_no_improve == patience:
        print("Arrêt précoce : La loss de validation ne s'améliore plus")
        break
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}")

Epoch 1, Validation Loss: 1.0005, Validation Accuracy: 0.7705
best model accuracy: 0.7705
Epoch 1, Loss: 1.4046227706330163
Epoch 2, Validation Loss: 0.7952, Validation Accuracy: 0.8165
best model accuracy: 0.8165
Epoch 2, Loss: 0.8944095905338015
Epoch 3, Validation Loss: 0.5909, Validation Accuracy: 0.8682
best model accuracy: 0.8682
Epoch 3, Loss: 0.722241191140243
Epoch 4, Validation Loss: 0.3957, Validation Accuracy: 0.8946
best model accuracy: 0.8946
Epoch 4, Loss: 0.5145852686039039
Epoch 5, Validation Loss: 0.2493, Validation Accuracy: 0.9384
best model accuracy: 0.9384
Epoch 5, Loss: 0.3314514181443623
Epoch 6, Validation Loss: 0.2068, Validation Accuracy: 0.9490
best model accuracy: 0.9490
Epoch 6, Loss: 0.23966027455670494
Epoch 7, Validation Loss: 0.1774, Validation Accuracy: 0.9565
best model accuracy: 0.9565
Epoch 7, Loss: 0.19342459738254547
Epoch 8, Validation Loss: 0.1534, Validation Accuracy: 0.9611
best model accuracy: 0.9611
Epoch 8, Loss: 0.16256227690194333
Epoch 

In [6]:
loss, accuracy = evaluate_model(model, validation_data_loader, loss_function, device, morph_to_ix)
print(f"Validation Accuracy : {accuracy:.4f}")
print(f"Validation loss : {loss:.4f}")

Validation Accuracy : 0.9669
Validation loss : 0.1338


<h2> testing models

In [11]:
sentences_test, morphs_test = load_data_preprocess("UD_French-Sequoia/fr_sequoia-ud-test.conllu", char_to_ix, max_word_len)

In [12]:
dataset_test = MORPHDataset(sentences_test, morphs_test, morph_to_ix, max_word_len,char_to_ix)
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [13]:
loss, accuracy = evaluate_model(model, data_loader_test, loss_function, device, morph_to_ix)
print(f"Validation Accuracy model: {accuracy:.4f}")
print(f"Validation loss model: {loss:.4f}")

In [ ]:
loss, accuracy = evaluate_model(best_model, data_loader_test, loss_function, device, morph_to_ix)
print(f"Validation Accuracy best model: {accuracy:.4f}")
print(f"Validation loss best model: {loss:.4f}")